Realizando imports e download de stopwords

In [ ]:
from gensim import corpora
from rank_bm25 import BM25Okapi
from nltk.corpus import stopwords
import string
import nltk

nltk.download('stopwords')

Carregando corpus

Pré-processamento

In [ ]:
def preprocessar(texto):
    tokens = texto.lower().translate(str.maketrans('', '', string.punctuation)).split()
    return [t for t in tokens if t not in stopwords.words('english')]

tokenized_docs = [preprocessar(doc) for doc in documentos]

Criando modelo BM25

In [ ]:
bm25 = BM25Okapi(tokenized_docs)

Testes de consulta

In [ ]:
consulta = "Who leads Brazil?"
consulta_tokens = preprocessar(consulta)
scores = bm25.get_scores(consulta_tokens)

In [ ]:
for idx, score in sorted(enumerate(scores), key=lambda x: -x[1])[:3]:
    print(f"[{score:.2f}] {documentos[idx]}")

Avaliação

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def calcular_precision_recall_f1(ranking, relevantes):
    precisions, recalls, f1s = [], [], []
    retrieved_relevant = 0
    total_relevant = len(relevantes)

    for k, doc_id in enumerate(ranking, start=1):
        if doc_id in relevantes:
            retrieved_relevant += 1
        precision = retrieved_relevant / k
        recall = retrieved_relevant / total_relevant if total_relevant > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)

    return precisions, recalls, f1s

def precision_11_points(precisions, recalls):
    recall_levels = np.linspace(0, 1.0, 11)
    precision_at_recall = []

    for r in recall_levels:
        precisions_at_r = [p for p, rc in zip(precisions, recalls) if rc >= r]
        if precisions_at_r:
            precision_at_recall.append(max(precisions_at_r))  # interpolated
        else:
            precision_at_recall.append(0.0)
    
    return recall_levels, precision_at_recall


In [ ]:
bm25_results = {}

for query in consultas:
    ranking = ranquear_bm25(query)
    relevantes = ground_truth[query]
    p, r, f1 = calcular_precision_recall_f1(ranking, relevantes)
    bm25_results[query] = (p, r, f1)

# Plotar curvas
plt.figure(figsize=(8, 6))
for i, query in enumerate(consultas):
    precisions, recalls, _ = bm25_results[query]
    recall_levels, prec_11 = precision_11_points(precisions, recalls)
    plt.plot(recall_levels, prec_11, label=f"BM25: Consulta {i+1}")

plt.xlabel("Recall")
plt.ylabel("Precision (Interpolado)")
plt.title("Curva Precision x Recall (BM25)")
plt.grid(True)
plt.legend()
plt.show()